In [1]:
shp = '/mnt/d/autocls_e3/classref/refexp32_vec.shp'

lulcmap = 'cosc2021'

expname = 'exp33'

yescls = 'roads'
nocls = 'no_roads'

outshp = '/mnt/d/autocls_e3/trainref/exp33_vec.shp'
outrst = '/mnt/d/autocls_e3/trainref/exp33_rst.tif'

db = 'gthruth_valeflores_v3'

refrst = '/mnt/d/autocls_e3/stdlmt/stdrst_valeflores.tif'

In [2]:
import numpy as np
import random as rnd
import pandas as pd

from glass.sql.q import q_to_obj
from glass.it.shp import dbtbl_to_shp
from glass.prop.prj import get_epsg
from glass.dtt.rst.torst import shp_to_rst

In [3]:
epsg = get_epsg(refrst)

In [4]:
ddf = q_to_obj(db, (
    "SELECT cls.pxid, cls.nameval, gt.class_code AS subcls "
    "FROM ("
    	"SELECT gtr.*, "
    	"CASE "
    		"WHEN foo.code IS NULL THEN 'mix' "
    		"ELSE foo.code "
    	"END AS class_code "
    	"FROM gthruth_fishnet AS gtr "
    	"LEFT JOIN ("
    		"SELECT pxc.pxid, mmcls.code, mmcls.classname "
    		"FROM pxclasses AS pxc "
    		"INNER JOIN ("
    			"SELECT mcls.classid, mcls.code, mcls.classname "
    			"FROM mapclasses AS mcls "
    			"INNER JOIN refmaps AS rm "
    			"ON mcls.mapfk = rm.mapid "
    			f"WHERE rm.slug = '{lulcmap}'"
    		") AS mmcls "
    		"ON pxc.clsid = mmcls.classid"
    	") AS foo "
    	"ON gtr.cid = foo.pxid"
    ") AS gt "
    "LEFT JOIN ("
    	"SELECT mm.pxid, jj.* "
    	"FROM pxexpclasses AS mm "
    	"INNER JOIN ("
    		"SELECT mj.expclsid, mj.codeval, mj.nameval, "
    		"jt.slug AS expslug "
    		"FROM expclasses AS mj "
    		"INNER JOIN experiments AS jt "
    		"ON mj.expfk = jt.eid "
    		f"WHERE jt.slug = '{expname}'"
    	") AS jj "
    	"ON mm.eclsid = jj.expclsid "
    ") AS cls "
    "ON gt.cid = cls.pxid "
    "WHERE gt.level = 1 "
    "ORDER BY cls.pxid"
))

In [5]:
ddf['subcls'] = np.where(
    ddf['nameval'] == 'roads',
    ddf['subcls'] + '+', ddf['subcls'] + '-'
)

In [6]:
# Give random numbers to each row
ddf['rnd'] = ddf.apply(lambda _: rnd.randint(1, 1000), axis=1)

# Scramble
ddf= ddf.sort_values(by=['subcls', 'rnd'])
ddf.reset_index(inplace=True)

In [7]:
ddf['cumaux'] = 1

ddf['cum'] = ddf.groupby('subcls')['cumaux'].transform(pd.Series.cumsum)
ddf['grpmax'] = ddf.groupby('subcls')['cum'].transform(pd.Series.max)

ddf['relcum'] = ddf.cum / ddf.grpmax * 100

In [8]:
train_df = ddf[ddf.relcum <= 30]

In [9]:
# Train to SHAPE

whr = ", ".join(train_df.pxid.astype(str).tolist())

q = (
    "SELECT jtbl.*, mtbl.geom "
    "FROM gthruth_fishnet AS mtbl "
    "LEFT JOIN ("
    	"SELECT mm.pxid, jj.* "
    	"FROM pxexpclasses AS mm "
    	"INNER JOIN ("
    		"SELECT mj.expclsid, mj.codeval, mj.nameval, "
    		"jt.slug AS expslug "
    		"FROM expclasses AS mj "
    		"INNER JOIN experiments AS jt "
    		"ON mj.expfk = jt.eid "
            f"WHERE jt.slug = '{expname}'"
    	") AS jj "
    	"ON mm.eclsid = jj.expclsid"
    ") AS jtbl "
    "ON mtbl.cid = jtbl.pxid "
    f"WHERE mtbl.cid = ANY(ARRAY[{whr}])"
)

dbtbl_to_shp(
    db, q, "geom", outshp,
    tableIsQuery=True, epsg=epsg
)

'/mnt/d/autocls_e3/trainref/exp33_vec.shp'

In [10]:
# Train to raster

shp_to_rst(
    outshp, 'codeval', None, 0,
    outrst, rst_template=refrst,
    api='gdal', rtype=int, dtype='UInt16'
)

'/mnt/d/autocls_e3/trainref/exp33_rst.tif'

In [11]:
ddf

,index,pxid,nameval,subcls,rnd,cumaux,cum,grpmax,relcum
0,44027,44028,roads,100+,1,1,1,5013,0.019948
1,56929,56930,roads,100+,1,1,2,5013,0.039896
2,20564,20565,roads,100+,2,1,3,5013,0.059844
3,36856,36857,roads,100+,2,1,4,5013,0.079793
4,46951,46952,roads,100+,2,1,5,5013,0.099741
...,...,...,...,...,...,...,...,...,...
61657,43053,43054,no_roads,mix-,1000,1,21779,21783,99.981637
61658,43430,43431,no_roads,mix-,1000,1,21780,21783,99.986228
61659,45118,45119,no_roads,mix-,1000,1,21781,21783,99.990819
61660,46751,46752,no_roads,mix-,1000,1,21782,21783,99.995409
